# # Operações Principais

* Ref.: https://odetocode.com/articles/739.aspx

* Ref.: https://learn.microsoft.com/pt-pt/samples/browse/?redirectedfrom=MSDN-samples


<div>
<img src="_img/linq1.png" width="500"/>
</div>

# Exemplo

In [ ]:
// Entities


// Classe Category
namespace Course.Entities {
    class Category {
        public int Id { get; set; }
        public string Name { get; set; }
        public int Tier { get; set; }
    }
}

// Classe Produc
using System.Globalization;
namespace Course.Entities {
    class Product {
        public int Id { get; set; }
        public string Name { get; set; }
        public double Price { get; set; }
        public Category Category { get; set; }

        public override string ToString() {
            return Id
                + ", "
                + Name
                + ", "
                + Price.ToString("F2", CultureInfo.InvariantCulture)
                + ", "
                + Category.Name
                + ", "
                + Category.Tier;
        }
    }
}

In [ ]:
using System;
using System.Linq;
using Course.Entities;
using System.Collections.Generic;

namespace Course {
    class Program {

        // Função auxiliar para imprimir
        static void Print<T>(string message, IEnumerable<T> collection) {
            Console.WriteLine(message);
            foreach (T obj in collection) {
                Console.WriteLine(obj);
            }
            Console.WriteLine();
        }

        // Código principal
        static void Main(string[] args) {

            // Criando categorias
            Category c1 = new Category() { Id = 1, Name = "Tools", Tier = 2 };
            Category c2 = new Category() { Id = 2, Name = "Computers", Tier = 1 };
            Category c3 = new Category() { Id = 3, Name = "Electronics", Tier = 1 };
            // Criando produtos
            List<Product> products = new List<Product>() {
                new Product() { Id = 1, Name = "Computer", Price = 1100.0, Category = c2 },
                new Product() { Id = 2, Name = "Hammer", Price = 90.0, Category = c1 },
                new Product() { Id = 3, Name = "TV", Price = 1700.0, Category = c3 },
                new Product() { Id = 4, Name = "Notebook", Price = 1300.0, Category = c2 },
                new Product() { Id = 5, Name = "Saw", Price = 80.0, Category = c1 },
                new Product() { Id = 6, Name = "Tablet", Price = 700.0, Category = c2 },
                new Product() { Id = 7, Name = "Camera", Price = 700.0, Category = c3 },
                new Product() { Id = 8, Name = "Printer", Price = 350.0, Category = c3 },
                new Product() { Id = 9, Name = "MacBook", Price = 1800.0, Category = c2 },
                new Product() { Id = 10, Name = "Sound Bar", Price = 700.0, Category = c3 },
                new Product() { Id = 11, Name = "Level", Price = 70.0, Category = c1 }
            };

            // OPERACOES COM LINQ

            // 1. Tier 1 e preço menor que 900
            // Dentro de where enviamos um func que retorna um bool. Via lambda.
            var r1 = products.Where(p => p.Category.Tier == 1 && p.Price < 900.0);
            Print("TIER 1 AND PRICE < 900:", r1); 
            // TIER 1 AND PRICE < 900:  
            // 6, Tablet, 700.00, Computers, 1  
            // 7, Camera, 700.00, Electronics, 1  
            // 8, Printer, 350.00, Electronics, 1  
            // 10, Sound Bar, 700.00, Electronics, 1

            // 2. Mostrar apenas nomes dos produtos da categoria Tools.
            var r2 = products.Where(p => p.Category.Name == "Tools").Select(p => p.Name);
            Print("NAMES OF PRODUCTS FROM TOOLS", r2);
            // NAMES OF PRODUCTS FROM TOOLS
            // Hammer
            // Saw
            // Level    

            // 3. Mostrar os campos que quero baseado em uma condição. No caso do nome da categoria defina um alias para não confundir com o nome do produto.
            var r3 = products.Where(p => p.Name[0] == 'C').Select(p => new { p.Name, p.Price, CategoryName = p.Category.Name });
            Print("NAMES STARTED WITH 'C' AND ANONYMOUS OBJECT", r3);

            // 4. Mostrar os produtos ordenados por preço e depois por nome.
            var r4 = products.Where(p => p.Category.Tier == 1).OrderBy(p => p.Price).ThenBy(p => p.Name);
            Print("TIER 1 ORDER BY PRICE THEN BY NAME", r4);
            
            // 5. Pega o resulado anterior ignorando os 2 primeiros e pegando os 4 seguintes.
            var r5 = r4.Skip(2).Take(4);
            Print("TIER 1 ORDER BY PRICE THEN BY NAME SKIP 2 TAKE 4", r5);

            // 6. Pega o primeiro elemento da lista.
            var r6 = products.FirstOrDefault();
            Console.WriteLine("First or default test1: " + r6);

            // 7. Pega o primeiro elemento da lista que atenda a condição.
            var r7 = products.Where(p => p.Price > 3000.0).FirstOrDefault();
            Console.WriteLine("First or default test2: " + r7); // First/Last/Single numa coleção vazia gera erro. Assim, usa o FirstOrDefault.
            Console.WriteLine();
            
            // 8. SingleOrDefault só funciona se a coleção tiver um único elemento que atenda a condição.
            // Consulta que retorna um único valor. Neste caso traz um objeo product e não um IEnumerable.
            var r8 = products.Where(p => p.Id == 3).SingleOrDefault();
            Console.WriteLine("Single or default test1: " + r8); // SingleOrDefault retorna null se não encontrar o elemento.   
            var r9 = products.Where(p => p.Id == 30).SingleOrDefault();
            Console.WriteLine("Single or default test2: " + r9); // Nulo
            Console.WriteLine();

            // 9. Agregação
            // Máximo da coleção com base no preço.
            var r10 = products.Max(p => p.Price);
            Console.WriteLine("Max price: " + r10);
            // Mínimo da coleção com base no preço. 
            var r11 = products.Min(p => p.Price);
            Console.WriteLine("Min price: " + r11);
            // Soma dos preços dos produtos da categoria 1. 
            var r12 = products.Where(p => p.Category.Id == 1).Sum(p => p.Price);
            Console.WriteLine("Category 1 Sum prices: " + r12);
            // Média dos preços dos produtos da categoria 1.    Retorna sequência de produtos.
            // Se a coleção for vazia, teremos um erro, pois não tem como fazer média dividindo por zero.
            var r13 = products.Where(p => p.Category.Id == 1).Average(p => p.Price);
            Console.WriteLine("Category 1 Average prices: " + r13);
            // Seleciona produtos da categoria 5, pega os preços e calcula a média. 
            // DefaultIfEmpty: Se a coleção até o selectt retornar vazia, retorna 0.0 antes mesmo de fazer o average.
            var r14 = products.Where(p => p.Category.Id == 5).Select(p => p.Price).DefaultIfEmpty(0.0).Average();
            Console.WriteLine("Category 5 Average prices: " + r14);
            // Select-Agregate ou Map-Reduce: Operação agregada personalizada.
            // Retorna a soma dos preços dos produtos da categoria 1. Veja que a função agregate recebe 2 parâmetros. 
            // O primeiro é o valor Default e o segundo é uma função que recebe 2 parâmetros e faz a soma deles.
            var r15 = products.Where(p => p.Category.Id == 1).Select(p => p.Price).Aggregate(0.0, (x, y) => x + y);
            Console.WriteLine("Category 1 aggregate sum: " + r15);
            Console.WriteLine();

            // 10. Agrupamento
            var r16 = products.GroupBy(p => p.Category);
            // Percorrer cada elemento do resultado r16. Depois vai imprimindo o título da categoria e em seguida seus produtos.
            foreach (IGrouping<Category, Product> group in r16) {
                Console.WriteLine("Category " + group.Key.Name + ":");
                foreach (Product p in group) {
                    Console.WriteLine(p);
                }
                Console.WriteLine();
            }
        }
    }
}